## imports & definitions

In [1]:
!pip install git+https://github.com/jbloomAus/SAEDashboard.git

  Cloning https://github.com/jbloomAus/SAEDashboard.git to /tmp/pip-req-build-8im300sy
  Running command git clone --filter=blob:none --quiet https://github.com/jbloomAus/SAEDashboard.git /tmp/pip-req-build-8im300sy
  Resolved https://github.com/jbloomAus/SAEDashboard.git to commit a5f8df15ef8619c4d08655e777d379a05b453346
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
DTYPE = 'bfloat16'
ctx_length = 128
subject = "college_math_cleaned"
import os

In [3]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append("/root/specialised-SAEs/")
from huggingface_hub import hf_hub_download
from sae_lens.sae import SAE
from sae_lens.training.training_sae import TrainingSAE
from sae_lens.training.activations_store import ActivationsStore
from sae_lens.config import LanguageModelSAERunnerConfig
from sae_lens.sae_training_runner import SAETrainingRunner
from load_sae_from_hf import load_sae_from_hf
from tqdm import tqdm
import gc
import torch
torch.set_grad_enabled(False)

In [4]:
# callum imports 
from IPython import get_ipython # type: ignore
ipython = get_ipython(); assert ipython is not None

# Standard imports
import torch
from datasets import load_dataset
import webbrowser
import os
from transformer_lens import utils, HookedTransformer
from datasets.arrow_dataset import Dataset
from huggingface_hub import hf_hub_download
import time

## Load stuff

In [5]:
model = HookedTransformer.from_pretrained_no_processing("gemma-2b-it", device="cuda", dtype=DTYPE)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b-it into HookedTransformer


In [6]:
from analysis_fns import get_owt_and_spec_tokens
owt_tokens, spec_tokens = get_owt_and_spec_tokens(model, f"jacobcd52/{subject}")
print(owt_tokens.shape, spec_tokens.shape)
print(model.tokenizer.batch_decode(spec_tokens[:2]))

owt_tokens has shape torch.Size([19962, 128])
total number of tokens: 2 million

spec_tokens has shape torch.Size([19868, 128])
total number of tokens: 2 million
torch.Size([19962, 128]) torch.Size([19868, 128])
['<bos> is ab and this fact, together with (4), implies that ab = ba, thereby completing the proof. 6.9. The ring Z. of integers modulo n is a field if the modulus is a prime (see Example 3.3). 6.10. According to Theorem 3.4.1 the system of rational numbers is a field. This is a restatement of parts (1)-(10) of that theorem. According to Theorem 3.6.1, the system of real numbers is a field. EXERCISES 6.1. Show that in the definition of a ring R', '<bos> variable called a. Alternatively,<eos> = can be used for assignment as in a = 2, but we will use \xad. We 683 684 Appendix B: Computations can then verify this assignment by simply typing a and hitting return, which causes the value of a to be printed. a [1] 2 Note that R is case sensitive, so A would be a different variable tha

In [7]:
l1 = 10
sae = load_sae_from_hf(f"jacobcd52/gemma-2b-it-ssae-{subject}",
                            f"gemma-2b-it_layer12_{subject}_l1={l1}_expansion=2_tokens=8192000_gsae_id=layer_12_stepan.safetensors",
                            f"gemma-2b-it_layer12_{subject}_l1={l1}_expansion=2_tokens=8192000_gsae_id=layer_12_stepan_cfg.json",
                            device="cuda", dtype=DTYPE           
)      

(…)8192000_gsae_id=layer_12_stepan_cfg.json:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

# Get dashboards

In [16]:
from sae_dashboard.sae_vis_data import SaeVisConfig
from sae_dashboard.sae_vis_runner import SaeVisRunner
from sae_dashboard.data_writing_fns import save_feature_centric_vis
import gc

for subject in  ["hs_bio_cleaned",
                "hs_math_cleaned", "hs_phys_cleaned", 
                "college_bio_cleaned", "college_math_cleaned", "college_phys_cleaned",
                "history_cleaned", "econ_cleaned"
                ]:
    owt_tokens, spec_tokens = get_owt_and_spec_tokens(model, f"jacobcd52/{subject}")
    sae = load_sae_from_hf(f"jacobcd52/gemma-2b-it-directsae-{subject}",
                            f"gemma-2b-it_layer12_{subject}_l1=2_expansion=2_tokens=8192000.safetensors",
                            f"gemma-2b-it_layer12_{subject}_l1=2_expansion=2_tokens=8192000_cfg.json",
                            device="cuda", dtype=DTYPE)
          
    feature_vis_config_gpt = SaeVisConfig(
        hook_point=sae.cfg.hook_name,
        features=list(range(100)),
        minibatch_size_features=128,
        minibatch_size_tokens=256,
        verbose=True,
        device="cuda",
    )

    visualization_data_gpt = SaeVisRunner(feature_vis_config_gpt).run(
        encoder=sae, # type: ignore
        model=model,
        tokens=spec_tokens[:10_000],  # type: ignore
)

    filename = f"dashboards/directsae_{subject}_spec.html"

    # create the file path if it doesn't exist
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    save_feature_centric_vis(sae_vis_data=visualization_data_gpt, filename=filename)


    feature_vis_config_gpt = SaeVisConfig(
        hook_point=sae.cfg.hook_name,
        features=list(range(100)),
        minibatch_size_features=128,
        minibatch_size_tokens=256,
        verbose=True,
        device="cuda",
    )

    visualization_data_gpt = SaeVisRunner(feature_vis_config_gpt).run(
        encoder=sae, # type: ignore
        model=model,
        tokens=owt_tokens[:10_000],  # type: ignore
)

    filename = f"dashboards/directsae_{subject}_owt.html"

    # create the file path if it doesn't exist
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    save_feature_centric_vis(sae_vis_data=visualization_data_gpt, filename=filename)

owt_tokens has shape torch.Size([19962, 128])
total number of tokens: 2 million

spec_tokens has shape torch.Size([19811, 128])
total number of tokens: 2 million


(…)2_expansion=2_tokens=8192000.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

(…)l1=2_expansion=2_tokens=8192000_cfg.json:   0%|          | 0.00/2.64k [00:00<?, ?B/s]

Forward passes to cache data for vis:   0%|          | 0/40 [00:00<?, ?it/s]

Extracting vis data from cached data:   0%|          | 0/100 [00:00<?, ?it/s]

...

Forward passes to cache data for vis:   0%|          | 0/40 [00:00<?, ?it/s]

Extracting vis data from cached data:   0%|          | 0/100 [00:00<?, ?it/s]

all_feat_acts dtype: torch.float32
all_feat_acts shape: torch.Size([10000, 128, 100])


┏━━━━━━┳━━━━━━┳━━━━━━━┓
┃ Task ┃ Time ┃ Pct % ┃
┡━━━━━━╇━━━━━━╇━━━━━━━┩
└──────┴──────┴───────┘

Saving feature-centric vis:   0%|          | 0/100 [00:00<?, ?it/s]